### Import Modules

In [1]:
import pandas as pd
import numpy as np
import pickle

### Load Datasets

In [2]:
DIR='/Users/tim/src/Metis/Project_3/data/raw/'
df_reign = pd.read_csv(DIR+'REIGN_2019_3.csv', encoding = 'ISO-8859-1')
df_wdi = pd.read_csv(DIR+'WDIData.csv')
df_penn = pd.read_csv(DIR+'pwt91.csv')

### Prepare REIGN Dataset

#### Filter

In [3]:
# Select relevant features
df_reign = df_reign[['country','year', 'government', 'militarycareer','tenure_months','prev_conflict', 'pt_attempt', 'exec_ant', 'irreg_lead_ant', 'exec_recent', 'lead_recent']]

# Select observations only between 1989 and 2016
df_r2 = df_reign[(df_reign['year'] > 1988) & (df_reign['year'] < 2017)]

In [4]:
#df_r2.info()

#### Aggregate Data by Year

In [5]:
# Create prev_year column
df_r2['prev_year'] = df_r2.year.shift()

# Convert tenure_months to tenure by year
df_r2.loc[(df_r2.prev_year.isnull() == True) | (df_r2.year != df_r2['prev_year']), 'tenure_months'] = (df_r2.tenure_months/12)
df_r2.loc[df_r2.year == df_r2.prev_year, 'tenure_months'] = 0 

# Sum up columns by country by year
df_r3 = df_r2.groupby(['country', 'year']).sum()

/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/metis/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


#### Flatten Groupby

In [6]:
df_r3.reset_index(inplace=True)

In [7]:
#df_r3.info()

#### Fix Aggregation

In [24]:
# Shift prev_conflict and coup attempts
df_r3['coup'] = df_r3.pt_attempt.shift()
df_r3['prev_year_conflict'] = df_r3.prev_conflict.shift()

# Convert categories back to binary form
df_r3.loc[df_r3.militarycareer > 0, 'militarycareer'] = 1 
df_r3.loc[df_r3.exec_ant > 0, 'exec_ant'] = 1
df_r3.loc[df_r3.irreg_lead_ant > 0, 'irreg_lead_ant'] = 1
df_r3.loc[df_r3.exec_recent > 0, 'exec_recent'] = 1
df_r3.loc[df_r3.lead_recent > 0, 'lead_recent'] = 1
df_r3.loc[df_r3.coup > 0, 'coup'] = 1

AttributeError: 'DataFrame' object has no attribute 'pt_attempt'

#### Rename variables

In [9]:
df_r3.rename(columns={'militarycareer':'mil_career',
                      'tenure_months':'tenure', 'prev_conflict':'prev_conflict2',
                      'prev_year_conflict':'prev_conflict'}, inplace=True)

#### Filter Again

In [10]:
# Remove obsolete columns
df_r3.drop(columns = ['prev_year', 'prev_conflict2', 'pt_attempt'], inplace=True)

# Select only observations from 1990 and on
df_r4 = df_r3[(df_r3['year'] >= 1990)]

In [11]:
# Convery binary categories to integers
df_r4.coup = df_r4.coup.astype(int)
df_r4.year = df_r4.year.astype(int)
df_r4.mil_career = df_r4.mil_career.astype(int)
df_r4.prev_conflict = df_r4.prev_conflict.astype(int)

/anaconda3/envs/metis/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [13]:
#df_r4.info()

### Prepare WDI Dataset

#### Filter

In [14]:
# Select country aggregated data
df_w2 = df_wdi[74965:]

# Drop Country Code, Indicator Name, and Unnamed Column
df_w2 = df_w2.drop(['Country Code', 'Indicator Name'], axis=1).drop(df_w2.columns[[-1]], axis=1).reset_index(drop=True)

#### Transform

In [15]:
# Unpivot by Country Name, Indicator Code, and Date
df_w3 = df_w2.melt(['Country Name', 'Indicator Code'], var_name='Date', value_name='Value')

# Combine Date and Country Columns
df_w3['Date-Country'] = df_w3['Date'] +'-'+ df_w3['Country Name']

#Drop Date and Country Columns
df_w3.drop(['Date', 'Country Name'], axis=1, inplace=True)

# Pivot by Date-Country and Indicator Code
df_w3 = df_w3.pivot(index='Date-Country', columns='Indicator Code', values='Value')
df_w3.reset_index(level=0, inplace=True)

#### Filter Again

In [16]:
# Select Relevant Features
df_w4 = df_w3[['Date-Country', 'NY.GDP.TOTL.RT.ZS', 'SN.ITK.DEFC.ZS', 'SE.XPD.TOTL.GD.ZS', 'SH.XPD.GHED.GD.ZS', 'FI.RES.TOTL.DT.ZS', 'GC.REV.XGRT.GD.ZS', 'EG.ELC.ACCS.ZS', 'DT.DFR.DPPG.CD', 'SP.DYN.LE00.IN', 'MS.MIL.XPND.GD.ZS', 'SH.STA.BASS.ZS', 'SH.H2O.BASW.ZS', 'GC.TAX.TOTL.GD.ZS', 'MS.MIL.TOTL.TF.ZS']]

# Split Date-Country Column 
df_w5 = pd.DataFrame(df_w4['Date-Country'].str.split("-", 1).tolist(), columns = ['year','country']).join(df_w4)

# Remove Date-Country Column
df_w5 = df_w5.drop(['Date-Country'], axis=1)

# Select Relevant Years
df_w5['year'] = df_w5['year'].apply(pd.to_numeric) 
df_w5 = df_w5[(df_w5['year'] >= 1990) & (df_w5['year'] < 2017)]

#### Rename Features

In [17]:
df_w5.rename(columns={'NY.GDP.TOTL.RT.ZS':'nat_resources', 'SN.ITK.DEFC.ZS':'food_scarcity',
                      'SE.XPD.TOTL.GD.ZS':'education', 'SH.XPD.GHED.GD.ZS':'healthcare',
                      'FI.RES.TOTL.DT.ZS':'reserves', 'GC.REV.XGRT.GD.ZS':'revenue',
                      'EG.ELC.ACCS.ZS': 'electricity', 'DT.DFR.DPPG.CD':'debt_forgiven', 
                      'SP.DYN.LE00.IN': 'life_exp', 'MS.MIL.XPND.GD.ZS': 'mil_spending', 
                      'SH.STA.BASS.ZS': 'sanitation', 'SH.H2O.BASW.ZS': 'water',
                      'GC.TAX.TOTL.GD.ZS':'tax', 'MS.MIL.TOTL.TF.ZS': 'military_pop'}, inplace=True)

In [18]:
#df_w5.info()

### Clean Penn Table Dataset

#### Filter

In [19]:
# Select relevant features
df_p2 = df_penn[['country', 'year', 'rgdpe', 'hc']]

# Select target years
df_p2 = df_p2[(df_p2['year'] >= 1990) & (df_p2['year'] < 2017)]

In [20]:
#df_p2.info()

### Join Datasets

In [21]:
df_merge2 = pd.merge(df_r4, df_w5, how='left', on=['country', 'year'])
df_merge3 = pd.merge(df_merge2, df_p2, how='left', on=['country', 'year'])

In [22]:
#df_merge3.info()

In [23]:
# filepath = '/Users/tim/src/Metis/Project_3/data/interim/df_merge3.pkl'
# with open(filepath, 'wb') as pkl:
#     pickle.dump(df_merge3, pkl)